## SETUP 

In [2]:
#import libraries
import pandas as pd
import sqlite3

In [4]:
#df = pd.DataFrame(#data here :/)

df = pd.DataFrame({'name': ['Juan', 'Victoria', 'Mary'], \
'age': [23, 34, 43], 'city': ['Miami', 'Buenos Aries', 'Santiago']})


In [5]:
df

,name,age,city
0,Juan,23,Miami
1,Victoria,34,Buenos Aries
2,Mary,43,Santiago


In [7]:
#We will sqlite3 library and create a connection
cnn = sqlite3.connect('jupyter_sql_tutorial.db')

In [11]:
df.to_sql('people', cnn)

In [12]:
%load_ext sql

The sql extension is already loaded. To reload it, use:
  %reload_ext sql


In [15]:
%sql sqlite:///jupyter_sql_tutorial.db

In [17]:
%%sql
SELECT *
FROM people

 * sqlite:///jupyter_sql_tutorial.db
Done.


index,name,age,city
0,Juan,23,Miami
1,Victoria,34,Buenos Aries
2,Mary,43,Santiago


In [18]:
%%sql
SELECT count(*)
FROM people

 * sqlite:///jupyter_sql_tutorial.db
Done.


count(*)
3


In [19]:
%%sql
SELECT sum(age) as 'age_sum'
FROM people

 * sqlite:///jupyter_sql_tutorial.db
Done.


age_sum
100


## Parameters walkthrough

In [20]:
#create dummy dataframe
df = pd.DataFrame({'transaction_id': ['9', '8', '7', '6', '5', '4', '3'], \
                  'user_id': ['rafa', 'roy', 'kenny', 'brendan', 'jurgen', 'roy', 'roy'],\
                  'transaction_date': ['2021-12-21', '2020-12-21', '2019-12-21',\
                                       '2018-11-21', '2017-10-21', '2019-03-02', '2010-01-01'],\
                  'amount': ['10', '15', '20', '24', '25', '31', '42']})

In [21]:
df

,transaction_id,user_id,transaction_date,amount
0,9,rafa,2021-12-21,10
1,8,roy,2020-12-21,15
2,7,kenny,2019-12-21,20
3,6,brendan,2018-11-21,24
4,5,jurgen,2017-10-21,25
5,4,roy,2019-03-02,31
6,3,roy,2010-01-01,42


In [22]:
#We will sqlite3 library and create a connection
cnn = sqlite3.connect('dummy.db')

In [29]:
df.to_sql('managers1', cnn)

In [30]:
%reload_ext sql

In [31]:
%sql sqlite:///dummy.db

In [32]:
%%sql
SELECT *
FROM managers1

 * sqlite:///dummy.db
Done.


index,transaction_id,user_id,transaction_date,amount
0,9,rafa,2021-12-21,10
1,8,roy,2020-12-21,15
2,7,kenny,2019-12-21,20
3,6,brendan,2018-11-21,24
4,5,jurgen,2017-10-21,25
5,4,roy,2019-03-02,31
6,3,roy,2010-01-01,42


In [34]:
%%sql
SELECT sum(amount) as 'spend_sum'
FROM managers1

 * sqlite:///dummy.db
Done.


spend_sum
167


In [35]:
%%sql
SELECT 
    user_id
    , count(*) as num_transactions
    , sum(amount) as total_amount
FROM
    managers1
WHERE
    user_id = 'roy'
    and transaction_date = '2019-03-02'

 * sqlite:///dummy.db
Done.


user_id,num_transactions,total_amount
roy,1,31


## For real this time

ovechal01, ovi8

1228 * 11,


In [3]:
stat_handle = input('stat handle:')
stat_sheet = f'./data/{stat_handle}_stats.csv'
stat_df = pd.read_csv(stat_sheet, sep= '\t', header= 0, index_col= None)

stat handle:ovechal01


In [4]:
stat_df

,date,goals,assists,+/-,pim,shots,shot%,TOI,hits,blocks,faceoff %
0,2005-10-05,2,0,1,2,5,40.0,17:36,NaN,NaN,NaN
1,2005-10-07,0,1,-2,0,1,0.0,17:44,NaN,NaN,NaN
2,2005-10-08,0,1,0,4,3,0.0,16:21,NaN,NaN,NaN
3,2005-10-10,1,0,1,2,6,16.7,15:03,NaN,NaN,NaN
4,2005-10-12,1,0,0,0,6,16.7,13:53,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
1223,2021-12-10,0,2,0,0,4,0.0,25:21,4.0,0.0,NaN
1224,2021-12-11,0,1,0,0,6,0.0,25:36,0.0,0.0,NaN
1225,2021-12-15,1,1,0,2,10,10.0,21:51,2.0,0.0,NaN
1226,2021-12-17,1,0,1,0,8,12.5,20:30,0.0,0.0,NaN


In [5]:
stat_handle = input('twitter handle:')
stat_sheet = f'./data/{stat_handle}.csv'
twitter_df = pd.read_csv(stat_sheet, sep= '\t', header= 0, index_col= None)

twitter handle:ovi8


In [6]:
twitter_df

,date time,display name,content
0,2021-12-02 18:19:51-05:00,Alex Ovechkin,The 2022 #CapsCanineCalendar is Ovi and Blake ...
1,2021-11-20 20:44:54-05:00,Alex Ovechkin,🏆 You took the biggest step!!! Congrats @WashS...
2,2021-11-16 04:04:50-05:00,Alex Ovechkin,Great to be part of the first @blockassetco Le...
3,2021-11-04 13:16:26-04:00,Alex Ovechkin,Good luck and see you guys soon in DC!! https:...
4,2021-10-28 22:15:31-04:00,Alex Ovechkin,Very proud to be a @Blockassetco Legend with @...
...,...,...,...
620,2009-01-21 14:51:20-05:00,Alex Ovechkin,At DC airport with tons of obama fans. Taking ...
621,2009-01-20 03:40:04-05:00,Alex Ovechkin,Lots of questions about my trick shot plans - ...
622,2009-01-20 01:35:55-05:00,Alex Ovechkin,Only thing better than scoring goals is winnin...
623,2009-01-19 04:31:30-05:00,Alex Ovechkin,Big win last night on a sick shot by Semin. No...


In [7]:
game_list = []
gm_dates = stat_df.iloc[:,0]
for gm in gm_dates:
    game_list.append(gm)
    
print(len(game_list))

1228


In [8]:
data_points = []
post_game = 0
tweet_dates = twitter_df.iloc[:,0]
tweet_times = twitter_df.iloc[:,1]
for gm, tm in zip(tweet_dates, tweet_times):
    if gm in game_list:
        
        hr = int(tm[:2])
        if hr < 19:
            data_points.append(gm)
        else:
            post_game += 1

print(f'{len(data_points)} good points of data, {post_game} postgame tweets')

0 good points of data, 0 postgame tweets


In [11]:
import pandas as pd

data_points = []
post_game = 0
tweet_dates = twitter_df.iloc[:,0]
#tweet_times = twitter_df.iloc[:,1]
for gm in tweet_dates:
    ymd = gm[:10]
    if ymd in game_list:
        gm_time = f'{ymd} 00:00:00-05:00'
        gm_time = pd.to_datetime(gm_time)
        gm = pd.to_datetime(gm)
        #hr = int(tm[:2])
        if gm < gm_time:
            data_points.append(gm)
        else:
            post_game += 1

print(f'{len(data_points)} good points of data, {post_game} postgame tweets')

4 good points of data, 91 postgame tweets


In [8]:
len(data_points)

0

Maybe need to include puck drop times? Would require altering stat scraper, and likely slowing it down signifigantly, to the magnitute of rendering an entire new webpage in beautfiul soup per row in the dataset. For Ovechkin this would include 1228 pages! Maybe I should use selenium as an alternative. However at least for Ovechkin, no gameday tweets were scheduled for 7pm EST. BRB Looking at tweet timestamps

In [9]:
for point in data_points:
    print(point)